# Some Experiments
used for some shit

## Imports

In [34]:
import sys
import os

# Füge das übergeordnete Verzeichnis zu sys.path hinzu
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

In [35]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from copy import deepcopy as dc

# own classes
from utilities import save_unscaled_sequential_data, load_sequential_time_series, train_test_split, extract_features_and_targets_reg, extract_features_and_targets, split_data_into_sequences, Scaler, ValidationLossAccumulationCallback, accuracy
from baseline_model.LSTM import LSTMRegression, train_model, LSTMClassification
from baseline_model.TimeSeriesDataset import TimeSeriesDataset
from baseline_model.mean_regressor import MeanRegressor
from data_generation.algorithmic.brownian_motion import brownian_motion
from data_generation.algorithmic.jittering import create_jittered_time_series

In [36]:
DATA_FOLDER = Path("./data")
SYNTHETIC_DATA_FOLDER = DATA_FOLDER / "synthetic"
REAL_DATA_FOLDER = DATA_FOLDER / "real"
LOAD_ORIGINAL_DATA = False

In [37]:
traffic_le = pd.read_csv(REAL_DATA_FOLDER / "metro_interstate_traffic_volume_label_encoded_no_categorical.csv")

## Jittering

In [38]:
jitter_factor = 0.05
jitt, noise = create_jittered_time_series(traffic_le, jitter_factor=jitter_factor, no_features=5)

In [39]:
jitt.to_csv(SYNTHETIC_DATA_FOLDER / f'mitv_jittered_{str(jitter_factor).replace(".", "")}.csv', index=False)

In [40]:
jitt.head()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all
0,287.442479,288.622573,0.030112,0.000836,1.878676
1,370.920706,289.359499,0.000000,0.000000,0.691748
2,269.677873,288.554392,0.307362,0.000601,0.959879
3,986.286712,287.650256,0.000000,0.000000,0.000000
4,2905.077609,288.414915,0.034302,0.000075,0.000000


In [41]:
traffic_le.head()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all
0,444,289.40,0.0,0.0,1
1,347,289.19,0.0,0.0,1
2,356,288.74,0.3,0.0,1
3,936,288.07,0.0,0.0,1
4,2886,287.87,0.0,0.0,1


In [42]:
jitt.describe()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all
count,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000
mean,3313.993927,282.687000,0.074687,0.000417,42.283954
std,1973.101318,12.387043,0.677228,0.008300,39.184933
min,0.000000,243.272209,0.000000,0.000000,0.000000
25%,1297.542364,273.437322,0.000000,0.000000,1.687612
50%,3515.540436,284.550130,0.001684,0.000003,38.964370
75%,4949.390680,292.824671,0.025839,0.000282,88.006129
max,7420.856445,310.590101,42.049093,0.510631,104.221390


In [43]:
traffic_le.describe()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all
count,28511.00000,28511.000000,28511.000000,28511.000000,28511.000000
mean,3313.74238,282.688768,0.061611,0.000250,42.122795
std,1971.53206,12.367361,0.678185,0.008298,39.316195
min,0.00000,243.390000,0.000000,0.000000,0.000000
25%,1289.00000,273.480000,0.000000,0.000000,1.000000
50%,3507.00000,284.550000,0.000000,0.000000,40.000000
75%,4948.00000,292.790000,0.000000,0.000000,90.000000
max,7280.00000,310.070000,42.000000,0.510000,100.000000


## Rescale timeGAN data

In [44]:
# save_unscaled_sequential_data(ori_data_path=REAL_DATA_FOLDER / "metro_interstate_traffic_volume_label_encoded.csv",
#                                      scaled_data_path=SYNTHETIC_DATA_FOLDER / "mitv_28499_12_11_lstm.csv",
#                                      scaled_data_shape=(28499, 12, 11),
#                                      no_features_to_inverse_scale=9)